# Advanced Features

## Introduction

When analyzing the real world datasets, we may have the following targets:

1. certain variables must be selected when some prior information is given;
2. selecting the weak signal variables when the prediction performance is mainly interested;
3. identifying predictors when group structure are provided;
4. pre-excluding a part of predictors when datasets have ultra high-dimensional predictors.

In the following content, we will illustrate the statistic methods to reach these targets in a one-by-one manner, and give quick examples to show how to perform the statistic methods in `abessLm` and the same steps can be implemented in all methods. Actually, in our methods, the targets can be properly handled by simply change some default arguments in the functions. 

## Nuisance Regression

Nuisance regression refers to best subset selection with some prior information that some variables are required to stay in the active set. For example, if we are interested in a certain gene and want to find out what other genes are associated with the response when this particular gene shows effect.

### Carrying Out the Nuisance Regression with `abess`

In the `abessLm()` (or other methods), the argument `always_select` is designed to realize this goal. User can pass a vector containing the indexes of the target variables to `always_select`. Here is an example.

In [38]:
import numpy as np
from abess.gen_data import gen_data
from abess.linear import abessLm

np.random.seed(0)

# gene data
n = 100
p = 20
k = 5
dt = gen_data(n = n, p = p, k = k, family = 'gaussian')
print('real coefficients:\n', dt.coef_, '\n')
print('real coefficients\' indexes:\n', np.nonzero(dt.coef_)[0])

real coefficients:
 [  0.           0.         115.01218243   0.           0.
  81.84924757   0.           0.           0.           0.
 104.77568224  64.30426355   0.           0.           0.
   0.           0.           0.         108.5408557    0.        ] 

real coefficients' indexes:
 [ 2  5 10 11 18]


In [39]:
model = abessLm(support_size = range(0, 6))
model.fit(dt.x, dt.y)
print('fitted coefficients:\n', model.coef_, '\n')
print('fitted coefficients\' indexes:\n', np.nonzero(dt.coef_)[0])

fitted coefficients:
 [  0.           0.         114.93792167   0.           0.
  81.8124385    0.           0.           0.           0.
 104.67076232  64.40856101   0.           0.           0.
   0.           0.           0.         108.73726174   0.        ] 

fitted coefficients' indexes:
 [ 2  5 10 11 18]


The coefficients are located in \[2, 5, 10, 11, 18\]. 
But if we suppose that the 7th and 8th variables are worthy to be included in the model, we can call:

In [40]:
model = abessLm(support_size = range(0, 6), always_select = [7, 8])
model.fit(dt.x, dt.y)
print('fitted coefficients:\n', model.coef_, '\n')
print('fitted coefficients\' indexes:\n', np.nonzero(dt.coef_)[0])

fitted coefficients:
 [  0.           0.         114.93792167   0.           0.
  81.8124385    0.           0.           0.           0.
 104.67076232  64.40856101   0.           0.           0.
   0.           0.           0.         108.73726174   0.        ] 

fitted coefficients' indexes:
 [ 2  5 10 11 18]


Now the variables we chosen are always in the model.

## Best group subset selection

Best group subset selection (BGSS) aims to choose a small part of non-overlapping groups to achieve the best interpretability on the response variable. BGSS is practically useful for the analysis of ubiquitously existing variables with certain group structures. For instance, a categorical variable with several levels is often represented by a group of dummy variables. Besides, in a nonparametric additive model, a continuous component can be represented by a set of basis functions (e.g., a linear combination of spline basis functions). Finally, specific prior knowledge can impose group structures on variables. A typical example is that the genes belonging to the same biological pathway can be considered as a group in the genomic data analysis.

The BGSS can be achieved by solving:

$$
    \min_{\beta\in \mathbb{R}^p} \frac{1}{2n} ||y-X\beta||_2^2,\quad s.t.\ ||\beta||_{0,r}\leq s 
$$

where $||\beta||_{0,r} = \sum_{j=1}^J I(||\beta_{G_j}||_2\neq 0)$ in which $||\cdot||_r$ is the $L_2$ norm and model size $s$ is a positive integer to be determined from data. Regardless of the NP-hard of this problem, Zhang et al develop a certifiably polynomial algorithm to solve it. This algorithm is integrated in the `abess` package, and user can handily select best group subset by assigning a proper value to the `group` arguments:

We still use the dataset `dt` generated before, which has 100 samples, 5 useful variables and 15 irrelevant varibales.

In [41]:
print('real coefficients:\n', dt.coef_, '\n')

real coefficients:
 [  0.           0.         115.01218243   0.           0.
  81.84924757   0.           0.           0.           0.
 104.77568224  64.30426355   0.           0.           0.
   0.           0.           0.         108.5408557    0.        ] 



Support we have some prior information that every 5 variables as a group:

In [42]:
group = np.linspace(0, 3, 4).repeat(5)
print('group index:\n', group)

group index:
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3.]


Then we can set the `group` argument in function. Besides, the `support_size` here indicates the number of groups, instead of the number of variables.

In [43]:
model = abessLm(support_size = range(0, 3))
model.fit(dt.x, dt.y, group = group)
print('coefficients:\n', model.coef_)

coefficients:
 [  4.07330876  14.02654966 133.63659942  -3.25926433  -8.02172721
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
  -4.14697258   1.53447211  16.29386214 112.43896075   8.85388996]


The fitted result suggest that only two groups are selected (since `support_size` is from 0 to 2) and the selected variables are shown before.

## Integrate SIS

Ultra-high dimensional predictors increase computational cost but reduce estimation accuracy for any statistical procedure. To reduce dimensionality from high to a relatively acceptable level, a fairly general asymptotic framework, named feature screening (sure independence screening) is proposed to tackle even exponentially growing dimension. The feature screening can theoretically maintain all effective predictors with a high probability, which is called "the sure screening property".

In our program, to carrying out the Integrate SIS, user need to set `is_screening = True` and pass an integer smaller than the number of the predictors to the `screening_size`. Then the program will first calculate the marginal likelihood of each predictor and reserve those predictors with the `screening_size` largest marginal likelihood. Then, the ABESS algorithm is conducted only on this screened subset. 

Here is an example.

In [44]:
n = 100
p = 1000
k = 3
np.random.seed(2)

# gene data
dt = gen_data(n = n, p = p, k = k, family = 'gaussian')
print('real coefficients\' indexes:', np.nonzero(dt.coef_)[0])

# fit
model = abessLm(support_size = range(0, 5), is_screening = True, screening_size = 100)
model.fit(dt.x, dt.y)
print('fitted coefficients\' indexes:', np.nonzero(model.coef_)[0])

real coefficients' indexes: [243 295 659]
fitted coefficients' indexes: [243 295 659]


## R tutorial

For R tutorial, please view [https://abess-team.github.io/abess/articles/v07-advancedFeatures.html](https://abess-team.github.io/abess/articles/v07-advancedFeatures.html).